# Issues / Improvements with OL

This document aims at collecting ideas for possible improvements of OL and differences between the current benchmark concerning vectortiles rendering which is the [vanilla mapbox](http://vectortiles.geo.admin.ch/gl-styles/ch.swisstopo.leichte-basiskarte.vt/v006/) viewer. The "model under test" is [vanilla OL](https://mf-geoadmin4.int.bgdi.ch/ol-minimalist/index.html) using `ol-mapbox-style` library to interpret mapbox style files and generate `VectorTilesLayer`s (see figure below). The style files are slightly modified to use only a single source host (`vectortiles100.geo.admin.ch`) instead of a range of five hosts (performance improvement on browsers supporting `http/2`). 

Conceptually there exist(ed) two different approaches to integrate a vector tiles layer using mapbox vector tiles in an OL map. In the first one - let's call it *ol native* - OL delegates only the task of layer creation and styling to a third party library called [ol-mapbox-style]() but takes care of layer rendering itself (using `CanvasVectorTileLayerRenderer`). In the second one - *ol + mapbox* - the mapbox-gl renderer is wrapped in an OL layer and hence treated like a "normal" OL layer, since (after a refactoring [PR #8922](https://github.com/openlayers/openlayers/pull/8922)) each layer is responsible for its rendering himself. 
![native ol vs. ol + mapbox-gl](ol-native-vs-ol+mapbox.dio.svg)
*native ol vs. ol + mapbox-gl*

A POC of this approach is available [here](https://codepen.io/pakb/pen/pBjGOP). It seems however that essentials parts have been removed from OL (maybe related to removal of WebGL parts) so this approach doesn't work with the current OL-`master` and is therefore not further evaluated in this document. Nevertheless, it seems a compelling idea to use mapbox-gl for rendering mapbox vectortiles as an OL layer. The possible limitations of this approach have to be evaluated though.

## Performance

We are comparing the performance of vanilla OL with vanilla MB rendering the vectortiles with the style produced by swisstopo Karto department. Performance improvements that could result from modifying / optimizing the style are explicitly noten taken into account here, MB renderer is able to cope with the style and amount of data in a performant way, so this is the "gold standard". 

- [timing of re-rendering and loading new tiles](#Timing-of-loading-new-tiles)
    
- [rendering](#Rendering)
    
- [caching](#Caching)


## Styling


[ol-mapbox-style](https://github.com/openlayers/ol-mapbox-style) provides decent interpretation of mapbox styles. However, there are still major and minor [differences](https://github.com/geoadmin/mf-geoadmin3/issues/4752). As mentioned in the [blog post](https://www.camptocamp.com/en/actualite/2018-openlayers-6-code-sprint/) about the Nov 2018 code sprint, an example has been developed to use the mapbox-gl renderer for an ol layer. 

- [ ] what is the status here?


## Documentation

- API documentation is very good
- "higher" level doc of concepts / architecture (e.g. call tree) is missing

## Roadmap

- in [ol wiki](https://github.com/openlayers/openlayers/wiki/OpenLayers-6:-call-for-funding) it is written that mapbox-gl should be integrated as ol-layer, status?
- how should several VT sources / layers be integrated in an app
    - basemap from one source with 5 vt-layers
    - additional source with 1-X vt-layers
    - raster layer
    
- 

## Resources

[[1]](https://docs.google.com/spreadsheets/d/1VZA57VBC37PWPP1vofD5HBXld2XgdsE92z1xdBN41bI/edit#gid=0) RE4_Crowdfunding_Acceptance_Protocol 

[[2]](https://docs.google.com/document/d/1CZndl2HZ8kuXAb7nHzMyCshT1zCBkKCJBRb6mSzy9Ok/edit#heading=h.3ec7yuydoshz) ol6 vt code sprint (goal and preparations)


--------

### Timing of loading new tiles

When recording performance of a mouse scroll zoom action it becomes apparent that the timing of loading new data is only when the zoom-animation has finished. 

![vanilla ol](img/ol-late-tile-loading.png)
*vanilla ol*

Comparing this to mapbox reveals that in MB, loading newer data is fired right after the zoom event has occured. Thus, tile loading has finished once animation has finished and rendering of the new data can immediately start. In what ways can this behaviour be improved in OL?

Ideas herefore could be:
- start loading of new tiles immediately: is this feasible, what would have to be changed in the architecture to not interfere with animation
- how is it determined, when new data has to be loaded? can this made predictive (e.g. panning left immediately start loading new tiles on the right of the map)

### Rendering
The overall rendering performance can't keep up with mapbox in general. 
#### Performance
According to [1], OL has been prepared to use webworkers. In a first step ([PR #8926](https://github.com/openlayers/openlayers/pull/8926)) execution planning and rendering has been separated. Is the [`offscreencanvas`](https://caniuse.com/#feat=offscreencanvas) feature necessary to make progress here? Browser-Support for offscreencanvas is still very limited and considering that we'll have to support older browser (certainly EDGE 17) for a while, are there other options? Workers currently seem to be used only in the Raster source.

In [1] (deliverables 1 and 2) WebGL is mentioned several times as full-fledged renderer, however with [PR #8951](https://github.com/openlayers/openlayers/pull/8951) most support for WebGL was dropped. What was the reason for this and could this be reconsidered? Performance analysis shows that MB is extensively using GPU.


#### Zoom steps

Zoom / pan animation is not done fully "vectorial" but based on rastered images that are zoomed. This results in several visual side effects
- feature size doesn't change smootly between zoom levels: features are enlarged too much by zooming the rastered image, subsequent rerendering reduces the feature size to the correct value. Vector zooming would allow for a smooth transition between feature sizes (this behaviour can be best observed at roundabouts).
- "fading in" of features when zooming is not working as intended (best observed in forest areas), MB zoom smoothly changes the forest color whereas OL zoom animation uses the "old" color and then abruptly changes to the new color.


Zooming by mouse wheel is currently too coarse (zooming step of one per wheel 'tick'), can be reduced to 0.33 (as in Mapbox) by reducing the `MAX_DELTA` constant (in `interaction/MouseWheelZoom.js:17`
```interaction/MouseWheelZoom.js:17
/**
 * Maximum mouse wheel delta.
 * @type {number}
 */
const MAX_DELTA = 0.3;
```

Furthermore, there is a difference in mouse wheel zooming and trackpad zooming. Trackpad shows a smoother zooming behaviour (more frequent re-rendering) than mouse wheel zooming. Performance recording shows that trackpad zooming consists of numerous small steps with intermediate re-rendering while mouse-wheel zooming is one long animation and subsequent re-rendering.

Note: the visual effect of the described behaviours can be slightly reduced by tweaking zoom steps and animation times.

### Caching

It seem that caching strategy of tiles could be improved. This is most apparent when zooming out of a region that was previously already displayed at a lower zoom level. Checking network traffic reveals that tiles are loaded again that have been previously loaded (although most of them from browser cache). Again, MB doesn't show this behaviour, when zooming in and out again at the exact same point, no tiles are reloaded.